# ECO758 Term Paper Script

In [1]:
import numpy as np
import pandas as pd
import statsmodels as stat
import matplotlib.pyplot as plt

In [2]:
gov = pd.read_excel('Data_Extract_From_Worldwide_Governance_Indicators.xlsx')
wdi = pd.read_excel('Data_Extract_From_World_Development_Indicators.xlsx')

In [3]:
gov = gov.dropna().drop(columns=['Series Code'])

In [4]:
wdi = wdi.iloc[0:80]

In [5]:
cty_list = list(wdi['Country Code'].unique())

### Filling missing values
Two steps: (i) Interpolate between missing values, (ii) replace missing values in front by backfill, (iii) replace missing values in the back by the last available value

In [6]:
pd.options.mode.chained_assignment = None  # default='warn'
k = pd.DataFrame()
for i in cty_list:
    m = wdi[wdi['Country Code'] == i].iloc[:,4:].interpolate(method='linear',axis =1).fillna(method = 'bfill',axis = 1)
    k = k.append(m)

In [7]:
wdi.iloc[:,4:] = k

#### Composite *gov* variable
All 5 are strongly correlated. Our composite factor is the average of all factors

In [8]:
k = pd.DataFrame()
for i in cty_list:
    m = gov[gov['Country Code']==i].sum(axis=0)
    k = pd.concat([k,m.to_frame().transpose()],axis=0)

In [9]:
k = k.reset_index(drop=True)

In [10]:
k['Country Code'] = cty_list
k['Country Name'] = ['Brunei Darussalam', 'Cambodia', 'Indonesia', 'Malaysia', 'Myanmar', 'Lao PDR', 'Philippines', 'Singapore', 'Vietnam', 'Thailand']
k['Series Name'] = 'Governance composite index'

In [11]:
gov = k

Descriptive Statistics

In [12]:
gov

,Country Name,Country Code,Series Name,2019 [YR2019],2018 [YR2018],2017 [YR2017],2016 [YR2016],2015 [YR2015],2014 [YR2014],2013 [YR2013],2012 [YR2012],2011 [YR2011],2010 [YR2010]
0,Brunei Darussalam,BRN,Governance composite index,3.59221,3.76862,3.43099,3.06085,3.38129,3.71173,3.785,3.73733,4.23775,4.28145
1,Cambodia,KHM,Governance composite index,-4.66956,-4.61633,-4.57166,-4.39027,-4.38164,-4.28967,-4.41846,-4.27206,-4.94871,-5.20811
2,Indonesia,IDN,Governance composite index,-0.996912,-0.883506,-1.03131,-1.06379,-1.76885,-1.31173,-2.02192,-2.29279,-2.68489,-2.91321
3,Malaysia,MYS,Governance composite index,2.57714,2.8874,1.66875,1.89261,2.30611,2.85862,2.13456,1.84126,1.71394,1.92027
4,Myanmar,MMR,Governance composite index,-5.69778,-5.57811,-5.34363,-4.95411,-6.91784,-7.15906,-7.82749,-8.29788,-9.68522,-10.4362
5,Lao PDR,LAO,Governance composite index,-4.77372,-4.59644,-4.27614,-4.09825,-4.23951,-4.09987,-4.88609,-5.1385,-5.7257,-6.00791
6,Philippines,PHL,Governance composite index,-1.84315,-2.07248,-2.02195,-2.07391,-1.4358,-1.15272,-1.81575,-2.25686,-2.72758,-3.23491
7,Singapore,SGP,Governance composite index,9.76874,9.81265,9.73764,9.64556,9.55308,9.39983,9.25789,9.40845,8.81347,8.79341
8,Vietnam,VNM,Governance composite index,-2.00266,-2.17062,-2.01415,-1.95104,-2.46976,-2.85122,-3.01559,-3.17087,-3.25362,-3.43923
9,Thailand,THA,Governance composite index,-1.19619,-1.71938,-1.61428,-1.89547,-1.96249,-1.8116,-1.7312,-1.58677,-1.65861,-2.1157


### Dropping Variable which has no data for a country

In [13]:
wdi = wdi[~wdi['Series Code'].str.contains('SE.SEC.ENRR')].reset_index(drop=True)

# Panel Data Analysis

In [18]:
from linearmodels import PanelOLS
from linearmodels import RandomEffects

In [108]:
%pip install linearmodels

Note: you may need to restart the kernel to use updated packages.


In [19]:
wdi.columns = ['Country Name', 'Country Code', 'Series Name', 'Series Code',
       '2019', '2018', '2017', '2016',
       '2015', '2014', '2013', '2012',
       '2011', '2010']

In [20]:
gov.columns = ['Country Name', 'Country Code', 'Series Name',
       '2019', '2018', '2017', '2016',
       '2015', '2014', '2013', '2012',
       '2011', '2010']
gov['Series Code'] = 'GOV.GOV'

In [21]:
wdi = wdi.append(gov)
wdi = wdi.reset_index(drop=True)

In [22]:
wdi.groupby('Country Code').head()

,Country Name,Country Code,Series Name,Series Code,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,Brunei Darussalam,BRN,GDP (constant 2010 US$),NY.GDP.MKTP.KD,1.4007e+10,1.34852e+10,1.34782e+10,1.33015e+10,1.36394e+10,1.36932e+10,1.40455e+10,1.43506e+10,1.42208e+10,1.37074e+10
1,Brunei Darussalam,BRN,Age dependency ratio (% of working-age populat...,SP.POP.DPND,38.5528,38.6956,39.012,39.245,39.2034,39.5728,39.8984,40.2422,40.7506,41.5404
2,Brunei Darussalam,BRN,"School enrollment, tertiary (% gross)",SE.TER.ENRR,31.4738,31.2118,33.565,31.9242,31.3006,32.0204,24.3401,22.4297,17.4043,15.4591
3,Brunei Darussalam,BRN,Trade (% of GDP),NE.TRD.GNFS.ZS,108.51,93.8963,85.1767,87.3183,89.8938,102.421,110.937,105.641,99.5365,95.3689
4,Brunei Darussalam,BRN,Gross fixed capital formation (% of GDP),NE.GDI.FTOT.ZS,38.507,40.8907,34.5875,34.3679,35.0264,27.2762,39.4585,32.7657,25.9151,23.5313
7,Cambodia,KHM,GDP (constant 2010 US$),NY.GDP.MKTP.KD,2.0921e+10,1.95424e+10,1.81842e+10,1.70218e+10,1.59036e+10,1.48582e+10,1.38677e+10,1.29174e+10,1.20371e+10,1.12423e+10
8,Cambodia,KHM,Age dependency ratio (% of working-age populat...,SP.POP.DPND,55.8044,55.6907,55.4805,55.3928,55.5727,55.8298,56.353,57.1259,58.0216,58.8814
9,Cambodia,KHM,"School enrollment, tertiary (% gross)",SE.TER.ENRR,14.739,13.6878,13.1343,13.135,13.1356,13.5742,14.0128,14.4515,14.8901,13.965
10,Cambodia,KHM,Trade (% of GDP),NE.TRD.GNFS.ZS,123.557,124.899,124.788,126.95,127.864,129.612,130.046,120.597,113.582,113.604
11,Cambodia,KHM,Gross fixed capital formation (% of GDP),NE.GDI.FTOT.ZS,23.3581,22.551,21.8977,21.6852,21.435,21.0398,18.946,17.3613,15.9718,16.1941


In [72]:
wdi.set_index('Country Name').set_index('Series Code')

,Country Code,Series Name,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
Series Code,,,,,,,,,,,,
NY.GDP.MKTP.KD,BRN,GDP (constant 2010 US$),1.4007e+10,1.34852e+10,1.34782e+10,1.33015e+10,1.36394e+10,1.36932e+10,1.40455e+10,1.43506e+10,1.42208e+10,1.37074e+10
SP.POP.DPND,BRN,Age dependency ratio (% of working-age populat...,38.5528,38.6956,39.012,39.245,39.2034,39.5728,39.8984,40.2422,40.7506,41.5404
SE.SEC.ENRR,BRN,"School enrollment, secondary (% gross)",92.0957,93.4828,92.9373,94.0542,96.4147,99.5916,104.913,106.265,101.431,99.1756
SE.TER.ENRR,BRN,"School enrollment, tertiary (% gross)",31.4738,31.2118,33.565,31.9242,31.3006,32.0204,24.3401,22.4297,17.4043,15.4591
NE.TRD.GNFS.ZS,BRN,Trade (% of GDP),108.51,93.8963,85.1767,87.3183,89.8938,102.421,110.937,105.641,99.5365,95.3689
...,...,...,...,...,...,...,...,...,...,...,...,...
GOV.GOV,LAO,Governance composite index,-4.77372,-4.59644,-4.27614,-4.09825,-4.23951,-4.09987,-4.88609,-5.1385,-5.7257,-6.00791
GOV.GOV,PHL,Governance composite index,-1.84315,-2.07248,-2.02195,-2.07391,-1.4358,-1.15272,-1.81575,-2.25686,-2.72758,-3.23491
GOV.GOV,SGP,Governance composite index,9.76874,9.81265,9.73764,9.64556,9.55308,9.39983,9.25789,9.40845,8.81347,8.79341


In [47]:
year = ['2019', '2018','2017','2016','2015','2014','2013','2012','2011','2010']

In [76]:
wdi.to_excel("panel_input.xlsx")

In [23]:
%pip install openpyxl
import openpyxl

Note: you may need to restart the kernel to use updated packages.


In [24]:
inp_new= pd.read_excel("panel_input.xlsx",sheet_name='Sheet2')

In [26]:
inp_new

,Column1,Country Name,Country Code,Series Name,Series Code,Year,Value
0,0,Brunei Darussalam,BRN,GDP (constant 2010 US$),NY.GDP.MKTP.KD,2019,1.400698e+10
1,0,Brunei Darussalam,BRN,GDP (constant 2010 US$),NY.GDP.MKTP.KD,2018,1.348522e+10
2,0,Brunei Darussalam,BRN,GDP (constant 2010 US$),NY.GDP.MKTP.KD,2017,1.347818e+10
3,0,Brunei Darussalam,BRN,GDP (constant 2010 US$),NY.GDP.MKTP.KD,2016,1.330146e+10
4,0,Brunei Darussalam,BRN,GDP (constant 2010 US$),NY.GDP.MKTP.KD,2015,1.363943e+10
...,...,...,...,...,...,...,...
795,79,Thailand,THA,Households and NPISHs Final consumption expend...,NE.CON.PRVT.KD,2014,1.974297e+11
796,79,Thailand,THA,Households and NPISHs Final consumption expend...,NE.CON.PRVT.KD,2013,1.963827e+11
797,79,Thailand,THA,Households and NPISHs Final consumption expend...,NE.CON.PRVT.KD,2012,1.945268e+11
798,79,Thailand,THA,Households and NPISHs Final consumption expend...,NE.CON.PRVT.KD,2011,1.812372e+11


In [27]:
inp_new = inp_new.drop(columns=['Column1'])

In [28]:
ser_code = list(pd.unique(inp_new['Series Code']))

In [29]:
q = inp_new

In [30]:
q_1 = q[q['Series Code']==ser_code[2]]

In [31]:
q_2 = pd.DataFrame()
for i in range(0,8):
    q_1 = q[q['Series Code']==ser_code[i]]
    k = q_1.rename(columns={'Value':f'{ser_code[i]}'}).drop(['Series Name', 'Series Code','Country Code'],axis = 1)
    k = k.reset_index(drop=True)
    q_2 = pd.concat([q_2,k],axis=1)

In [32]:
k = q_1.rename(columns={'Value':f'{ser_code[5]}'}).drop(['Series Name', 'Series Code','Country Code'],axis = 1)

In [33]:
q_2

,Country Name,Year,NY.GDP.MKTP.KD,Country Name,Year,SP.POP.DPND,Country Name,Year,GOV.GOV,Country Name,...,NE.TRD.GNFS.ZS,Country Name,Year,NE.GDI.FTOT.ZS,Country Name,Year,BX.KLT.DINV.WD.GD.ZS,Country Name,Year,NE.CON.PRVT.KD
0,Brunei Darussalam,2019,1.400698e+10,Brunei Darussalam,2019,38.552772,Brunei Darussalam,2019,3.592210,Brunei Darussalam,...,108.509658,Brunei Darussalam,2019,38.507027,Brunei Darussalam,2019,2.771141,Brunei Darussalam,2019,2.776375e+09
1,Brunei Darussalam,2018,1.348522e+10,Brunei Darussalam,2018,38.695628,Brunei Darussalam,2018,3.768617,Brunei Darussalam,...,93.896320,Brunei Darussalam,2018,40.890729,Brunei Darussalam,2018,3.804741,Brunei Darussalam,2018,2.620802e+09
2,Brunei Darussalam,2017,1.347818e+10,Brunei Darussalam,2017,39.011953,Brunei Darussalam,2017,3.430995,Brunei Darussalam,...,85.176749,Brunei Darussalam,2017,34.587522,Brunei Darussalam,2017,3.858208,Brunei Darussalam,2017,2.564949e+09
3,Brunei Darussalam,2016,1.330146e+10,Brunei Darussalam,2016,39.244999,Brunei Darussalam,2016,3.060851,Brunei Darussalam,...,87.318267,Brunei Darussalam,2016,34.367871,Brunei Darussalam,2016,-1.320522,Brunei Darussalam,2016,2.449261e+09
4,Brunei Darussalam,2015,1.363943e+10,Brunei Darussalam,2015,39.203444,Brunei Darussalam,2015,3.381287,Brunei Darussalam,...,89.893790,Brunei Darussalam,2015,35.026437,Brunei Darussalam,2015,1.324702,Brunei Darussalam,2015,2.481957e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Thailand,2014,3.825263e+11,Thailand,2014,39.730776,Thailand,2014,-1.811595,Thailand,...,130.905525,Thailand,2014,24.661397,Thailand,2014,1.221452,Thailand,2014,1.974297e+11
96,Thailand,2013,3.787974e+11,Thailand,2013,39.531279,Thailand,2013,-1.731195,Thailand,...,132.462274,Thailand,2013,25.383538,Thailand,2013,3.791269,Thailand,2013,1.963827e+11
97,Thailand,2012,3.688836e+11,Thailand,2012,39.381895,Thailand,2012,-1.586768,Thailand,...,137.674939,Thailand,2012,26.993653,Thailand,2012,3.244565,Thailand,2012,1.945268e+11
98,Thailand,2011,3.439706e+11,Thailand,2011,39.231021,Thailand,2011,-1.658605,Thailand,...,139.675407,Thailand,2011,25.836369,Thailand,2011,0.667087,Thailand,2011,1.812372e+11


In [34]:
q_2 = q_2.reset_index(drop=True)

In [35]:
q_2.columns.values[0] = 'Country_Name'
q_2.columns.values[1] = 'year'

In [36]:
q_2

,Country_Name,year,NY.GDP.MKTP.KD,Country Name,Year,SP.POP.DPND,Country Name,Year,GOV.GOV,Country Name,...,NE.TRD.GNFS.ZS,Country Name,Year,NE.GDI.FTOT.ZS,Country Name,Year,BX.KLT.DINV.WD.GD.ZS,Country Name,Year,NE.CON.PRVT.KD
0,Brunei Darussalam,2019,1.400698e+10,Brunei Darussalam,2019,38.552772,Brunei Darussalam,2019,3.592210,Brunei Darussalam,...,108.509658,Brunei Darussalam,2019,38.507027,Brunei Darussalam,2019,2.771141,Brunei Darussalam,2019,2.776375e+09
1,Brunei Darussalam,2018,1.348522e+10,Brunei Darussalam,2018,38.695628,Brunei Darussalam,2018,3.768617,Brunei Darussalam,...,93.896320,Brunei Darussalam,2018,40.890729,Brunei Darussalam,2018,3.804741,Brunei Darussalam,2018,2.620802e+09
2,Brunei Darussalam,2017,1.347818e+10,Brunei Darussalam,2017,39.011953,Brunei Darussalam,2017,3.430995,Brunei Darussalam,...,85.176749,Brunei Darussalam,2017,34.587522,Brunei Darussalam,2017,3.858208,Brunei Darussalam,2017,2.564949e+09
3,Brunei Darussalam,2016,1.330146e+10,Brunei Darussalam,2016,39.244999,Brunei Darussalam,2016,3.060851,Brunei Darussalam,...,87.318267,Brunei Darussalam,2016,34.367871,Brunei Darussalam,2016,-1.320522,Brunei Darussalam,2016,2.449261e+09
4,Brunei Darussalam,2015,1.363943e+10,Brunei Darussalam,2015,39.203444,Brunei Darussalam,2015,3.381287,Brunei Darussalam,...,89.893790,Brunei Darussalam,2015,35.026437,Brunei Darussalam,2015,1.324702,Brunei Darussalam,2015,2.481957e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Thailand,2014,3.825263e+11,Thailand,2014,39.730776,Thailand,2014,-1.811595,Thailand,...,130.905525,Thailand,2014,24.661397,Thailand,2014,1.221452,Thailand,2014,1.974297e+11
96,Thailand,2013,3.787974e+11,Thailand,2013,39.531279,Thailand,2013,-1.731195,Thailand,...,132.462274,Thailand,2013,25.383538,Thailand,2013,3.791269,Thailand,2013,1.963827e+11
97,Thailand,2012,3.688836e+11,Thailand,2012,39.381895,Thailand,2012,-1.586768,Thailand,...,137.674939,Thailand,2012,26.993653,Thailand,2012,3.244565,Thailand,2012,1.945268e+11
98,Thailand,2011,3.439706e+11,Thailand,2011,39.231021,Thailand,2011,-1.658605,Thailand,...,139.675407,Thailand,2011,25.836369,Thailand,2011,0.667087,Thailand,2011,1.812372e+11


In [37]:
q_2 = q_2.drop(columns=['Country Name','Year'])

In [38]:
exog_vars = [ser_code[1], ser_code[2],ser_code[3],ser_code[4],ser_code[5],ser_code[6],ser_code[7]]

In [39]:
q_2 = q_2.set_index(['Country_Name', 'year'])

In [40]:
q_2 = q_2 + 10

In [41]:
q_2 = q_2.apply(np.log)

In [46]:
rank_check=False
mod = PanelOLS(q_2[ser_code[0]],exog)

c:\users\siddharth\appdata\local\programs\python\python38\lib\site-packages\linearmodels\shared\exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [45]:
import statsmodels.api as sm
exog = sm.add_constant(q_2[exog_vars])

In [54]:
print(mod.fit())

                          PanelOLS Estimation Summary                           
Dep. Variable:         NY.GDP.MKTP.KD   R-squared:                        0.9944
Estimator:                   PanelOLS   R-squared (Between):              0.9978
No. Observations:                  99   R-squared (Within):               0.6350
Date:                Tue, Apr 20 2021   R-squared (Overall):              0.9944
Time:                        01:24:55   Log-likelihood                    72.568
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      2293.5
Entities:                          10   P-value                           0.0000
Avg Obs:                       9.9000   Distribution:                    F(7,91)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             2293.5
                            

In [55]:
mod = RandomEffects(q_2[ser_code[0]],exog)

c:\users\siddharth\appdata\local\programs\python\python38\lib\site-packages\linearmodels\shared\exceptions.py:35: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [56]:
print(mod.fit())
# Random Effect

                        RandomEffects Estimation Summary                        
Dep. Variable:         NY.GDP.MKTP.KD   R-squared:                        0.9438
Estimator:              RandomEffects   R-squared (Between):              0.9694
No. Observations:                  99   R-squared (Within):               0.9057
Date:                Tue, Apr 20 2021   R-squared (Overall):              0.9691
Time:                        01:25:14   Log-likelihood                    143.39
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      218.13
Entities:                          10   P-value                           0.0000
Avg Obs:                       9.9000   Distribution:                    F(7,91)
Min Obs:                       9.0000                                           
Max Obs:                      10.0000   F-statistic (robust):             216.10
                            

In [81]:
exog_var = ['1','2','3']

In [ ]:
exog = sm.add_constant(inp_new[exog_var])

In [57]:
print("Descriptive Statistics")
print(q_2.describe())

Descriptive Statistics
       NY.GDP.MKTP.KD  SP.POP.DPND    GOV.GOV  SE.TER.ENRR  NE.TRD.GNFS.ZS  \
count      100.000000   100.000000  99.000000   100.000000      100.000000   
mean        25.398822     4.032083   2.075328     3.703423        4.728422   
std          1.550501     0.177696   0.590957     0.410857        0.602411   
min         22.687267     3.610662  -1.155878     3.141316        2.322424   
25%         23.472812     3.919672   1.745000     3.323991        4.280549   
50%         26.044219     4.052056   2.084119     3.713076        4.793020   
75%         26.553690     4.185965   2.487149     3.919473        5.001607   
max         27.817069     4.343520   2.986321     4.593972        5.963833   

       NE.GDI.FTOT.ZS  BX.KLT.DINV.WD.GD.ZS  NE.CON.PRVT.KD  
count      100.000000            100.000000      100.000000  
mean         3.596015              2.735542       24.703786  
std          0.131551              0.325260        1.667822  
min          3.257010     